In [1]:
%load_ext autoreload
%autoreload 2

import pathlib
from datetime import timedelta
from typing import List, Dict
import random

import pandas as pd
pd.set_option('display.max_rows', 200)
import numpy as np

from tqdm import tqdm

import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp

from data_handlers.runtimes import *

In [2]:
experiment_path = pathlib.Path(f"/home/sterz/maci-docker-compose/maci_data/binary_files")
experiment_ids = [5] #[4, 5, 6, 7]

In [6]:
event_frame = parse_bundle_events(experiment_path, experiment_ids)

Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/5/621
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/5/458
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/5/522
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/5/555
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/5/452
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/5/425
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/5/528
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/5/619
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/5/613
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/5/589
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/5/567
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/5/510
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/5/484
Parsing /home/sterz/maci-docker-compose/maci_data/binary_files/5/583
Parsing /home/sterz/maci-docker-co

/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/

Parsing done


/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)
/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/runtimes.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  time_df = time_df.append(instance)


In [7]:
for bundle, group in event_frame[(event_frame["Event"] == "creation")].groupby("Bundle"):
    if len(group) != 1:
        print(bundle)
        print(group)

dtn://n1/-729075638361-0

       Simulation ID  Payload Size               Timestamp     Event Node  \
78952            448        100000 2023-02-07 09:00:38.405  creation   n1   
251599           449        100000 2023-02-07 09:00:38.405  creation   n1   

                            Bundle Software  Bundles per Second   CLA  \
78952   dtn://n1/-729075638361-0\n   DTN7Rs                   1  http   
251599  dtn://n1/-729075638361-0\n   DTN7Rs                  10  mtcp   

        # Nodes  # Payloads     Relative Timestamp  
78952         2         100 0 days 00:01:13.216000  
251599        2         100 0 days 00:00:02.144000  
